In the previous section we introduced `Enum` and `Sum`.   In this section we will give a more thoruogh introduction ADTs.

ADTs in peak do not have a fixed representation in bitvectors.  Instead this representation is later defined seperately with an *assembler*.  This allows a designer to fix the semenatics procossor with respect an abstract instruction without defining the exact bit level encoding.  This makes it simple to extend the instruction set as one need not edit existing code.

In [1]:
import hwtypes as hw

Enums define a set of named abstract values.  They are inherited from `hw.Enum`.

Enum members may be assigned a value which acts as suggestion to the assembler or may be left entirely abstract using `hw.Enum.Auto` or its alias `new_instruction`.  Enum members suppot equals and not equals.  

In [ ]:
class Opcode(hw.Enum):
    

Tuple documentation

In [2]:
# Tuple

In [ ]:
# class T(enum): ...

# class Ext(enum, extends=T): ... # copy elements of T into Ext

# T < Ext

#from hwtypes import Sum
#T = Sum[hw.Bit, hw.BitVector[4]]
#S = Sum[hw.Bit, hw.BitVector[4], hw.BitVector[6]]
#issubclass(S, T)